In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt
from hw3 import io
from hw3 import smith_waterman
from hw3 import optimize
from hw3 import visualize
from sklearn import metrics
import random
import copy

In [2]:
def byFirst(elem):
    return elem[0]

## Question 2

#### 2.1. I implemented particle swarm optimization in "optimize.py"
#### 2.2. Below: Optimization of BLOSUM50 matrix.

In [3]:
#####import blosum matrix
blosums = []
blosum_files = ['BLOSUM50']
for file in blosum_files:
    importme = 'blosums/'+file
    blosums.append(io.import_blosum(importme))
###################
scoring_matrix = blosums[0]
FPRs = [0.0,0.1,0.2,0.3]
pen_gap_extend = -5
pen_gap_open = -6
true_pos = io.import_pairs('data/Pospairs.txt')
true_neg = io.import_pairs('data/Negpairs.txt')
###see what the fitness is prior to starting
optimize.calc_fitness(FPRs,blosums[0],true_pos,true_neg,pen_gap_open,pen_gap_extend)

2.12

### The fitness for PAM100 before optimization is: 
## 2.02

In [ ]:
#######
particles = 50
iterations = 10
###################
optimized_output = optimize.particle_swarm(scoring_matrix,FPRs,pen_gap_extend,pen_gap_open,true_pos,true_neg,particles,iterations)



### The fitness after optimization is:
## 2.54

### I will now compare the ROC curves of each matrix at true postive rate of 0.7

In [ ]:
####readin
true_pos = io.import_pairs('data/Pospairs.txt')
true_neg = io.import_pairs('data/Negpairs.txt')
all_pairs = [true_pos,true_neg]
#####import blosum matrices
blosums = []
blosum_files = ['BLOSUM50']
for file in blosum_files:
    importme = 'blosums/'+file
    blosums.append(io.import_blosum(importme))
blosums.append(copy.deepcopy(optimized_output[1]))
blosum_files.append('BLOSUM50 OPTIMIZED')
#########calculate
scores_array = []
######
for scoring_matrix in blosums:
    score = []
    for file in all_pairs:
        for pair in file:
            ###this contains just the scores
            output = smith_waterman.smith_waterman(pair[0],pair[1],scoring_matrix,pen_gap_open,pen_gap_extend)
            score.append([output[2],pair])
    scores_array.append(score)
#######
threshold = 0.7
all_FPRs = []
#####
for scores in scores_array:
    FPR,TPR,AUC = visualize.calc_ROC(true_pos, true_neg, scores)
    for tpr in TPR:
        if tpr >= threshold:
            all_FPRs.append(FPR[TPR.index(tpr)])
            break





# ####initialize
# total_confusions = []
# pen_gap_extend = -5
# pen_gap_open = -6
# plt.figure(figsize=(10,10))
# #####################
# #####loop through scoring matrices and calculate confusion for each one
# #####################
# matrix_num = 0
# line_widths = [5,2]
# for scoring_matrix in blosums:
#     total_scores = []
#     ########do alignments for all pairs
#     for file in all_pairs:
#         for pair in file:
#             ###this contains just the scores
#             output = smith_waterman.smith_waterman(pair[0],pair[1],scoring_matrix,pen_gap_open,pen_gap_extend)
#             total_scores.append([output[2],pair])
#     #####determine true positives by TPR = 0.7
#     ###sort, largest to smallest
#     total_scores.sort(key=byFirst,reverse=True)
#     ####loop through and count TPR until matches TPR = 0.7
#     FPR = 0.3
#     FP = 0
#     TP = 0
#     TPR = 0.0
#     actual_FPR = -1
#     predicted_pos = []
#     predicted_neg = []
#     found_all_positives = False
#     for alignment in total_scores:
#         tmp_rate = 0
#         if found_all_positives == False:
#             if alignment[1] not in true_pos: 
#                 #####evaluate new FPR
#                 new_FPR = float(FP+1)/float(len(true_neg))
#                 #####evaluate if this will exceed FPR threshold
#                 ###this is used to handle small or 0 values of FPR
#                 if new_FPR >= FPR:
#                     found_all_positives = True
#                     if FPR == 0:
#                         predicted_neg.append(alignment)
#                     else:
#                         predicted_pos.append(alignment)
#                 else:
#                     FP += 1
#                     actual_FPR = float(FP)/float(len(true_neg))
#                     predicted_pos.append(alignment)
#             else:
#                 TP += 1
#                 TPR = float(TP)/float(len(true_pos))
#                 predicted_pos.append(alignment)
#             if actual_FPR == FPR:
#                 found_all_positives = True
#         else:
#             predicted_neg.append(alignment)   
#     ####prepare for plotting ROC curves
#     label = []
#     pred = []
#     ###by counting
#     TP = 0
#     FP = 0
#     TN = 0
#     FN = 0
#     ###
#     for alignment in total_scores:
#         if alignment[1] in true_pos:
#             label.append(1)
#             TP += 1
#         else:
#             label.append(0)
#             TN += 1
#     ###
#     for alignment in total_scores:
#         if alignment in predicted_pos:
#             pred.append(1)
#         else:
#             pred.append(0)
#     ##
#     FP = len(predicted_pos)-TP
#     FN = len(predicted_neg)-TN
#     #####plot
#     fpr, tpr, thresh = metrics.roc_curve(label, pred)
#     auc = metrics.roc_auc_score(label, pred)
#     ###
#     plt.plot(fpr,tpr,label=blosum_files[matrix_num]+" auc="+str(auc),linewidth=line_widths[matrix_num%2],alpha=0.5)
#     #####save confusion matrix
#     confusion = [blosum_files[matrix_num],[TP,FP,TN,FN]]
#     total_confusions.append(confusion)
#     #####
#     matrix_num +=1
# ########
# plt.plot([0, 1], [0, 1],'r--')
# plt.legend(loc=0)
# plt.suptitle('Fig 6. ROC curve for PAM100 blosum matrix before and after matrix optimization.')
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

In [ ]:
visualize.plot_ROCs(true_pos, true_neg, scores_array,blosum_files)
print('Fig 3. ROC curves by BLOSUM matrix')

### Discussion

## Q2.3.
#### Optimzation of MATIO scoring matrix

In [ ]:
#####import blosum matrix
blosums = []
blosum_files = ['MATIO']
for file in blosum_files:
    importme = 'blosums/'+file
    blosums.append(io.import_blosum(importme))
###################
scoring_matrix = blosums[0]
FPRs = [0.0,0.1,0.2,0.3]
pen_gap_extend = -5
pen_gap_open = -6
true_pos = io.import_pairs('data/Pospairs.txt')
true_neg = io.import_pairs('data/Negpairs.txt')
###see what the fitness is prior to starting
calc_fitness(FPRs,blosums[0],true_pos,true_neg,pen_gap_open,pen_gap_extend)

### The fitness for MATIO before optimization is: 
## 2.02

In [ ]:
particles = 30
iterations = 5
###################
optimized_output = optimize.particle_swarm(scoring_matrix,FPRs,pen_gap_extend,pen_gap_open,true_pos,true_neg,particles,iterations)



### The fitness after optimization is:
## 2.40

### I will now compare the ROC curves of each matrix at true postive rate of 0.7

#### 2.3. Below: Optimization of MATIO 100 matrix (prior best).

In [ ]:
####readin
true_pos = io.import_pairs('data/Pospairs.txt')
true_neg = io.import_pairs('data/Negpairs.txt')
all_pairs = [true_pos,true_neg]
#####import blosum matrices
blosums = []
blosum_files = ['MATIO']
for file in blosum_files:
    importme = 'blosums/'+file
    blosums.append(io.import_blosum(importme))
blosums.append(copy.deepcopy(optimized_output[1]))
blosum_files.append('MATIO OPTIMIZED')
####initialize
total_confusions = []
pen_gap_extend = -5
pen_gap_open = -6
plt.figure(figsize=(10,10))
#####################
#####loop through scoring matrices and calculate confusion for each one
#####################
matrix_num = 0
line_widths = [5,2]
for scoring_matrix in blosums:
    total_scores = []
    ########do alignments for all pairs
    for file in all_pairs:
        for pair in file:
            ###this contains just the scores
            output = smith_waterman.smith_waterman(pair[0],pair[1],scoring_matrix,pen_gap_open,pen_gap_extend)
            total_scores.append([output[2],pair])
    #####determine true positives by TPR = 0.7
    ###sort, largest to smallest
    total_scores.sort(key=byFirst,reverse=True)
    ####loop through and count TPR until matches TPR = 0.7
    TP = 0
    predicted_pos = []
    predicted_neg = []
    found_all_positives = False
    for alignment in total_scores:
        if found_all_positives == False:
            predicted_pos.append(alignment)
            if alignment[1] in true_pos: 
                TP += 1
                TPR = float(TP)/float(len(true_pos))
            if TPR == 0.7:
                found_all_positives = True
        else:
            predicted_neg.append(alignment)   
    ####prepare for plotting ROC curves
    label = []
    pred = []
    ###by counting
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    ##
    for alignment in predicted_pos:
        pred.append(1)
        if alignment[1] in true_pos:
            label.append(1)
            TP += 1
        else:
            label.append(0)
            FP += 1
    ##
    for alignment in predicted_neg:
        pred.append(0)
        if alignment[1] in true_neg:
            label.append(0)
            TN += 1
        else:
            label.append(1)
            FN += 1
    #####plot
    fpr, tpr, thresh = metrics.roc_curve(label, pred)
    auc = metrics.roc_auc_score(label, pred)
    ###
    plt.plot(fpr,tpr,label=blosum_files[matrix_num]+" auc="+str(auc),linewidth=line_widths[matrix_num%2],alpha=0.5)
    #####save confusion matrix
    confusion = [blosum_files[matrix_num],[TP,FP,TN,FN]]
    total_confusions.append(confusion)
    #####
    matrix_num +=1
########
plt.plot([0, 1], [0, 1],'r--')
plt.legend(loc=0)
plt.suptitle('Fig 7. ROC curve for MATIO blosum matrix before and after matrix optimization.')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Now let's look at the confusion matrices between the two alignment scoring matrices:


In [ ]:
for confusion in total_confusions:
    print(confusion)

### Q2.3

#### 2.4. Future improvements for optimization algorithm.

#### I have implemented a modified particle swarm algorithm. I wrote the particle swarm algorithm to handle BLOSUM matrices specifically with the following features:
    Makes matrices symetrical
    Calculates using my smith-waterman algorithm
    Introduces a jitter for the scoring matrix at each iteration
#### I chose this algorithm because:
    It is applicable to any function
    Intuitive design and implementation
    low number of parameters to test (which I did on small scale).
    Searches a global space quickly and is not easily caught in local minima.
#### I implemented a small jitter for the scoring matrix because I observed in my testing that the particle with highest fitness often failed to reach a higher fitness level. After I implemented the jitter, oftentimes that particle reached an even higher level of jitter by randomizing the matrix by small quantities. Because the particle swarm memorizes (for each particle) its previous best position, I reasoned that if the jitter was detrimental, it would revert back to its previous best position.

#### To improve this algorithm I would implement a "reproducing" particle swarm algorithm. At each iteration I would take the highest fitness particle, make copies of it with significant quantities of change in each copy, and then eliminate the lowest fitness particles. In this way I would combine the benefits of the particle swarm algorithm, specifically how it searches a global space quickly, but also combine the benefits of local optimization. By taking an optimized solution and then searching around that solution, I can search the global space first and then sample the local space once a candidate region with high fitness has been found.

## Q2.5 

#### Necessary features for a global 

In [ ]:
#########determine true positives by FPR in {0.0,0.1,0.2,0.3}

####
total_confusions = []
####
pen_gap_extend = -5
pen_gap_open = -6
#####################
#####loop through scoring matrices and calculate confusion for each one
#####################
for scoring_matrix in blosums:
    total_scores = []
    ########do alignments for all pairs
    for file in all_pairs:
        for pair in file:
            ###this contains just the scores
            output = smith_waterman.smith_waterman(pair[0],pair[1],scoring_matrix,pen_gap_open,pen_gap_extend)
            total_scores.append([output[2],pair])
    ##########
    
    ###sort, largest to smallest
    total_scores.sort(key=byFirst,reverse=True)
    ####loop through and count TPR until matches TPR = 0.7
    TP = 0
    predicted_pos = []
    predicted_neg = []
    found_all_positives = False
    for alignment in total_scores:
        if found_all_positives == False:
            predicted_pos.append(alignment)
            if alignment[1] in true_pos: 
                TP += 1
                TPR = float(TP)/float(len(true_pos))
            if TPR == 0.7:
                found_all_positives = True
        else:
            predicted_neg.append(alignment)   
    ####prepare for plotting ROC curves
    label = []
    pred = []
    ###by counting
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    ##
    for alignment in predicted_pos:
        pred.append(1)
        if alignment[1] in true_pos:
            label.append(1)
            TP += 1
        else:
            label.append(0)
            FP += 1
    ##
    for alignment in predicted_neg:
        pred.append(0)
        if alignment[1] in true_neg:
            label.append(0)
            TN += 1
        else:
            label.append(1)
            FN += 1
    #####plot
    fpr, tpr, thresh = metrics.roc_curve(label, pred)
    auc = metrics.roc_auc_score(label, pred)
    ###
    fig = plt.plot(fpr,tpr,label=blosum_files[matrix_num]+" auc="+str(auc),linewidth=line_widths[matrix_num%2],alpha=0.5)
    #####save confusion matrix
    confusion = [blosum_files[matrix_num],[TP,FP,TN,FN]]
    total_confusions.append(confusion)

########